MovieLensを利用してサンプルを作成する  
MovieLensのダウンロード

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.dirname(Path().resolve()))
from modules.movie_lens import MovieLensDataset
from modules.matrix_factorization import MatrixFactorization

/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file system 

In [4]:
movielens_datasets = MovieLensDataset()

In [5]:
movielens_datasets.ratings.head()

,userId,movieId,rating,timestamp,userIdx,movieIdx
0,1,307,3.5,1256677221,0,304
1,6,307,4.0,832059248,5,304
2,56,307,4.0,1383625728,55,304
3,71,307,5.0,1257795414,70,304
4,84,307,3.0,999055519,83,304


In [6]:
from collections import Counter
from sklearn.model_selection import train_test_split

In [7]:
# 学習データと検証データの用意
train, test = train_test_split(movielens_datasets.ratings, test_size=0.3, shuffle=True, random_state=42)
print(train.shape, test.shape)

(19427410, 6) (8326034, 6)


In [10]:
# モデルの作成
matrix_factorization = MatrixFactorization()
user_num = len(movielens_datasets.users)
item_num = len(movielens_datasets.movies)
model = matrix_factorization.generate_model(user_num=user_num, item_num=item_num, dimension=100)

In [11]:
matrix_factorization.train(
    model=model, 
    train_x=[train['userIdx'].values, train['movieIdx'].values],
    train_y=train['rating'].values,
    test_x=[test['userIdx'].values, test['movieIdx'].values],
    test_y=test['rating'].values
)

Epoch 1/200
18973/18973 [==============================] - 2445s 129ms/step - loss: 2.0102 - root_mean_squared_error: 1.4178 - val_loss: 1.2443 - val_root_mean_squared_error: 1.1155
Epoch 2/200
18973/18973 [==============================] - 2525s 133ms/step - loss: 1.1335 - root_mean_squared_error: 1.0647 - val_loss: 1.0663 - val_root_mean_squared_error: 1.0326
Epoch 3/200
18973/18973 [==============================] - 2580s 136ms/step - loss: 1.0147 - root_mean_squared_error: 1.0073 - val_loss: 0.9878 - val_root_mean_squared_error: 0.9939
Epoch 4/200
18973/18973 [==============================] - 2577s 136ms/step - loss: 0.9487 - root_mean_squared_error: 0.9740 - val_loss: 0.9337 - val_root_mean_squared_error: 0.9663
Epoch 5/200
18973/18973 [==============================] - 2590s 137ms/step - loss: 0.8962 - root_mean_squared_error: 0.9467 - val_loss: 0.8887 - val_root_mean_squared_error: 0.9427
Epoch 6/200
18973/18973 [==============================] - 2620s 138ms/step - loss: 0.8534

KeyboardInterrupt: 

In [12]:
model.save('../datasets/matrix-factorization-100dim.h5')

In [13]:
model.predict([np.array([283214, 283214, 283214]), np.array([1, 2, 3])], verbose=1)

1/1 [==============================] - 0s 52ms/step


array([[3.8641486],
       [3.7219005],
       [3.1750238]], dtype=float32)